In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [3]:
df = pd.read_csv('train.csv')
df.shape

(848835, 6)

In [4]:
df.head()

,row_id,time,x,y,direction,congestion
0,0,1991-04-01 00:00:00,0,0,EB,70
1,1,1991-04-01 00:00:00,0,0,NB,49
2,2,1991-04-01 00:00:00,0,0,SB,24
3,3,1991-04-01 00:00:00,0,1,EB,18
4,4,1991-04-01 00:00:00,0,1,NB,60


In [5]:
df = df.drop('time', axis = 1)
df = df.drop('row_id', axis = 1)

In [6]:
df = pd.get_dummies(df, drop_first=True)

In [7]:
df.head()

,x,y,congestion,direction_NB,direction_NE,direction_NW,direction_SB,direction_SE,direction_SW,direction_WB
0,0,0,70,0,0,0,0,0,0,0
1,0,0,49,1,0,0,0,0,0,0
2,0,0,24,0,0,0,1,0,0,0
3,0,1,18,0,0,0,0,0,0,0
4,0,1,60,1,0,0,0,0,0,0


In [8]:
X = pd.read_csv('X.csv', index_col=[0])
y = pd.read_csv('y.csv', index_col=[0])

In [9]:
print(df.shape)
print(X.shape)
print(y.shape)

(848835, 10)
(4330, 2145)
(4330, 1)


In [10]:
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit()

In [11]:
for train_index, test_index in tscv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244

In [12]:
X_test[0:1]

,0,1,2,3,4,5,6,7,8,9,...,2135,2136,2137,2138,2139,2140,2141,2142,2143,2144
3609,1.0,3.0,86.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,3.0,51.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
print(X.shape)
print(X_test.shape)
print(y.shape)
print(y_test.shape)

(4330, 2145)
(721, 2145)
(4330, 1)
(721, 1)


In [14]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(max_depth=10, random_state=0)
rf.fit(X, y)
rf_pred = rf.predict(X_test)

C:\Users\rujek\AppData\Local\Temp\ipykernel_19556\1400285206.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X, y)


In [15]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
print(r2_score(y_test, rf_pred))
print(mean_absolute_error(y_test, rf_pred))
print(mean_squared_error(y_test, rf_pred))

0.88399085206559
4.2853398332439445
30.805238491788888


In [16]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X, y)
lin_pred = reg.predict(X_test)

In [17]:
print(r2_score(y_test, lin_pred))
print(mean_absolute_error(y_test, lin_pred))
print(mean_squared_error(y_test, lin_pred))

0.7496500027975659
6.005141513522885
66.47830371618602


In [18]:
from xgboost import XGBRegressor
xgbst = XGBRegressor()
xgbst.fit(X, y)
lin_xgb = xgbst.predict(X_test)

C:\Users\rujek\Anaconda3\envs\strive\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
C:\Users\rujek\Anaconda3\envs\strive\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [19]:
print(r2_score(y_test, lin_xgb))
print(mean_absolute_error(y_test, lin_xgb))
print(mean_squared_error(y_test, lin_xgb))

0.9886938567091444
1.318720277900008
3.002249794077116


In [20]:
from sklearn.svm import SVR
svr_model = SVR()
svr_model.fit(X, y)
svr_pred = svr_model.predict(X_test)

C:\Users\rujek\Anaconda3\envs\strive\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [21]:
print(r2_score(y_test, svr_pred))
print(mean_absolute_error(y_test, svr_pred))
print(mean_squared_error(y_test, svr_pred))

0.5683045709212922
8.543066292153645
114.63303442331558


In [22]:
#gridsearch
#from sklearn.model_selection import GridSearchCV
#parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
#clf = GridSearchCV(svr_model, parameters)
#clf.fit(X, y)
#sorted(clf.cv_results_.keys())

In [23]:
rang = y_train.values.max() - y_train.values.min()
rang

100

In [24]:
regressors = {
  "Linear Regression": LinearRegression(),
  "Random Forest": RandomForestRegressor(max_depth=10, random_state=0),
  "XGBoost": XGBRegressor(learning_rate =0.1, n_estimators=1000, max_depth=5, gamma=0, subsample=0.8),
  "SVR": SVR(C=1.0, epsilon=0.2),
}

In [25]:
from sklearn import metrics

results = pd.DataFrame({'Model': [], 'MSE': [], 'MAE': [], 'r2_score': [], " % error": []})

for model_name, model in regressors.items():
    model.fit(X, y)
    pred = model.predict(X_test)
    
    results = results.append({"Model": model_name,
                            "MSE": mean_squared_error(y_test, pred),
                            "MAE": mean_absolute_error(y_test, pred),
                            "r2_score": r2_score(y_test, pred),
                            " % error": mean_squared_error(y_test, pred) / rang},
                            ignore_index=True)


results_ord = results.sort_values(by=['MSE'], ascending=True, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['MSE', 'MAE'], vmin=0, vmax=100, color='#5fba7d')

print(results_ord)

C:\Users\rujek\AppData\Local\Temp\ipykernel_19556\3339750750.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"Model": model_name,
C:\Users\rujek\AppData\Local\Temp\ipykernel_19556\3339750750.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\rujek\AppData\Local\Temp\ipykernel_19556\3339750750.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"Model": model_name,
C:\Users\rujek\Anaconda3\envs\strive\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (

               Model         MSE       MAE  r2_score   % error
1            XGBoost    0.031305  0.133719  0.999882  0.000313
2      Random Forest   30.805238  4.285340  0.883991  0.308052
3  Linear Regression   66.478304  6.005142  0.749650  0.664783
4                SVR  114.684092  8.547577  0.568112  1.146841


C:\Users\rujek\AppData\Local\Temp\ipykernel_19556\3339750750.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"Model": model_name,


In [26]:
# predict next 12 hours